In [1]:
from flask import Flask, request, jsonify
import mysql.connector
import json

app = Flask(__name__)

def connect_db():
    return mysql.connector.connect(
        host='localhost',
        user='your_username',
        password='your_password',
        database='your_database'
    )

@app.route('/register', methods=['POST'])
def register_user():
    data = request.json
    user_name = data['username']
    password = data['password']
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('INSERT INTO Users (User_name, Password) VALUES (%s, %s)', (user_name, password))
    conn.commit()
    cursor.close()
    conn.close()
    return jsonify({'message': f'User {user_name} registered successfully.'})

@app.route('/login', methods=['POST'])
def login_user():
    data = request.json
    user_name = data['username']
    password = data['password']
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Users WHERE User_name = %s AND Password = %s', (user_name, password))
    user = cursor.fetchone()
    cursor.close()
    conn.close()
    
    if user:
        return jsonify({'message': 'Login successful', 'user_info': user})
    else:
        return jsonify({'message': 'User not found'}), 404

@app.route('/add_task', methods=['POST'])
def add_task():
    data = request.json
    user_name = data['username']
    tid = data['tid']
    arr = json.dumps(data.get('arr', []))
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT User_id FROM Users WHERE User_name = %s', (user_name,))
    user_id = cursor.fetchone()

    if user_id:
        cursor.execute('INSERT INTO Data (Uid, Tid, Arr, Res) VALUES (%s, %s, %s, NULL)', (user_id[0], tid, arr))
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({'message': 'Task added successfully'})
    else:
        cursor.close()
        conn.close()
        return jsonify({'message': 'User not found'}), 404

@app.route('/user_tasks', methods=['GET'])
def get_user_tasks():
    user_name = request.args.get('username')
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT Tid FROM Data WHERE Uid = (SELECT User_id FROM Users WHERE User_name = %s)', (user_name,))
    tasks = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return jsonify({'tasks': [task[0] for task in tasks]})

@app.route('/task_info', methods=['GET'])
def get_task_info():
    user_name = request.args.get('username')
    tid = request.args.get('tid')
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT Arr, Res FROM Data WHERE Uid = (SELECT User_id FROM Users WHERE User_name = %s) AND Tid = %s', (user_name, tid))
    task_info = cursor.fetchone()
    cursor.close()
    conn.close()
    
    if task_info:
        return jsonify({'Arr': json.loads(task_info[0] or '[]'), 'Res': task_info[1]})
    else:
        return jsonify({'message': 'Task not found'}), 404

@app.route('/update_task', methods=['PUT'])
def update_task():
    data = request.json
    user_name = data['username']
    tid = data['tid']
    arr = json.dumps(data.get('arr', []))
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('UPDATE Data SET Arr = %s WHERE Uid = (SELECT User_id FROM Users WHERE User_name = %s) AND Tid = %s', (arr, user_name, tid))
    conn.commit()
    cursor.close()
    conn.close()

    return jsonify({'message': 'Task updated successfully'})

@app.route('/update_result', methods=['PUT'])
def update_result():
    data = request.json
    user_name = data['username']
    tid = data['tid']
    res = data['res']
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('UPDATE Data SET Res = %s WHERE Uid = (SELECT User_id FROM Users WHERE User_name = %s) AND Tid = %s', (res, user_name, tid))
    conn.commit()
    cursor.close()
    conn.close()

    return jsonify({'message': 'Result updated successfully'})
if __name__ == '__main__':
    app.run(debug=True)